# 1. Data Balancing

# 2. Data Split


# 3. Classification

### (a) Decision Tree

### (b) AdaBoost

### (c) K-Nearest Neighbors (K-NN)

### (d) Random Forests


### (e) Na¨ıve Bayes


# 4. Model Parameter Tuning


# 5. Report Requirements

# 6. Bonus